In [ ]:
import psycopg2
import psycopg2.extras
from sqlalchemy import create_engine 
import numpy as np
import pandas as pd

In [ ]:
conn = psycopg2.connect(dbname="nyc_taxi", user="postgres", password="123456789")

cur = conn.cursor( )

In [ ]:
addr_df=pd.read_csv('dataset/yellow_tripdata_2020-01.csv', low_memory=False)
addr_df.reset_index(drop=True,inplace=True)
addr_df.head()

In [ ]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS nyc_cab_data;
        CREATE UNLOGGED TABLE nyc_cab_data (
            VendorID         INTEGER,
            tpep_pickup_datetime               TIMESTAMP,
            tpep_dropoff_datetime          TIMESTAMP,
            passenger_count           DECIMAL,
            trip_distance       DECIMAL,
            RatecodeID       DECIMAL,
            store_and_fwd_flag       TEXT,
            PULocationID       INTEGER,
            DOLocationID       INTEGER,
            payment_type       DECIMAL,
            fare_amount       DECIMAL,
            extra       DECIMAL,
            mta_tax       DECIMAL,
            tip_amount       DECIMAL,
            tolls_amount       DECIMAL,
            improvement_surcharge       DECIMAL,
            total_amount       DECIMAL,
            congestion_surcharge       DECIMAL
        );""")

In [ ]:
with conn.cursor() as cursor:
    create_staging_table(cursor)

In [ ]:
def send_csv_to_psql(connection,csv,table_):
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open(csv, "r")
    table = table_
    with connection.cursor() as cur:
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur.copy_expert(sql=sql % table, file=file)
        connection.commit()
#         cur.close()
#         connection.close()
    return connection.commit()

send_csv_to_psql(conn,'dataset/yellow_tripdata_2020-01.csv','nyc_cab_data')